In [1]:
import pandas as pd
import numpy as np
import math
import missingno as mn
import matplotlib.pyplot as plt

In [2]:
#want to check the nearest couple of tube stops as well as walking distance
#could just stick to primary for now and expand to top couple later

In [3]:
#read in the pubs dataset already processed with distance to their nearest tube line attached
#easiest thing is probably to start adding total travel time to nearest node point of a different line
#again can cover that later, for now just take the closest one
#orrrr just treat them as independant pubs and double the length of your data, it wouldn't add much processing time
%store -r pubswithdist
#read in the already calculated tube network travel time data set
%store -r data_tubetravel
data_stations = pd.read_csv("C:\\Users\\George\\Downloads\\MITM\\stations_csv.sv.csv")
#now grab the all important travel times
data_travel = pd.read_csv("C:\\Users\\George\\Downloads\\MITM\\travel_times.csv")

In [4]:
#need to assign every pub a nearest tube
#need to define a function for distance between coordinates
#use the haversine formula
#need the angle in radians

def coords_to_distance(x1,x2,y1,y2):
    x1 = x1 * math.pi / 180
    x2 = x2 * math.pi / 180
    y1 = y1 * math.pi / 180
    y2 = y2 * math.pi / 180
    dlat = x2 - x1
    dlon = y2 - y1
    a = (math.pow(math.sin(dlat/2),2)) + math.cos(x1) * math.cos(x2) * (math.pow(math.sin(dlon/2),2))
    c = 2 * math.atan2(math.sqrt(a),math.sqrt(1-a))
    d = c * 6373
    #this is in km
    return d

In [5]:
def find_second_station(previous_station):
    alreadycheckedlines= []
    for line in data_travel[data_travel['station1'] == previous_station].iterrows():
        #print(line)
        alreadycheckedlines.append(line[1]['line'])
        datatravel_temp = data_travel[~data_travel['line'].isin(alreadycheckedlines)]
        data_filtered = data_stations[data_stations['id'].isin(datatravel_temp['station1'])]
    return data_filtered

In [129]:
#find_second_station(120)

#data_stations.head(10)

In [6]:
def distance_to_pubs(input_lat,input_long,user_num,data_stations):
    min_distance = 1000
    closest_line_id = []
    closest_line = []
    for line in data_stations.itertuples():
        current_dist = coords_to_distance(getattr(line,'latitude'),input_lat,getattr(line,'longitude'),input_long)
        if current_dist < min_distance:
            min_distance = current_dist
            closest_line_id = getattr(line,'id')
            #closest_line = getattr(line, '')
    
    #this loop returns the closest tube station to the input coords
    #then the below works out the travel time to all the things from that
    
    x= 1
    person_travel = []
    tube_id = []
    tube_results = []
    for line in data_tubetravel[data_tubetravel['id'] == closest_line_id]['travel_times']:
        while x < len(data_stations):
            person_travel.append(line[x]['time']) 
            tube_id.append(x)
            x += 1

    person_travel = pd.DataFrame(person_travel)
    tube_id = pd.DataFrame(tube_id)
    individ_tube = pd.merge(tube_id,person_travel,left_index = True, right_index = True)
    individ_tube.columns = ['tube_id','travel_time_to_tube']
    

    full_travel = pd.merge(pubswithdist,individ_tube, how = 'left',left_on = '0_y',right_on = 'tube_id')
    min_distance_time = min_distance / 0.084
    full_travel['total time' + user_num] = full_travel['traveltime_totube_pub'] + full_travel['travel_time_to_tube'] + min_distance_time
    return full_travel,closest_line_id

In [7]:
#input_lat = 51.5020275
#input_long = -0.0267862
numberofpeople = 2
full_travel, previous_closest = distance_to_pubs(51.5020275,-0.0267862,'1',data_stations)
full_travel_testing,previous_closest = distance_to_pubs(51.5020275,-0.0267862,'1_2ndclosest',find_second_station(previous_closest))
full_travel_testing = pd.DataFrame(full_travel_testing['total time1_2ndclosest'])
testing = pd.merge(full_travel,full_travel_testing,left_index= True,right_index = True)

#forgot to take into account walking for tube2
    
#need to input closest tube id somehow

#landmark east
full_travel2,previous_closest = distance_to_pubs(51.4860931,-0.2587167,'2',data_stations)
full_travel2 = pd.DataFrame(full_travel2['total time2'])
#chiswick
#need to generalise this to extend to extra people


In [8]:
def minimise_total_travel(input_set):
#    while x < numberofpeople:
#        columns_to_sum.append('total time' + str(x)) 
    input_set['total'] = input_set['first and second'] + input_set['total time2']
    #input_set2 = input_set.copy()
    input_set = input_set.sort_values(['total'])
    return input_set.head(10)

In [9]:
count = 0
for line in testing:
    testing['first and second'][count] = min(testing['total time1'][count],testing['total time1_2ndclosest'][count])
    count += 1

KeyError: 'first and second'

In [144]:
actualfulltravel = pd.merge(testing,full_travel2,left_index = True, right_index =True)

In [146]:
minimise_total_travel(actualfulltravel)

,index,0_x,1,2,3,4,8,latitude,longitude,0_y,0,traveltime_totube_pub,tube_id,travel_time_to_tube,total time1,total time1_2ndclosest,first and second,total time2,total
2278,11340,120221,The Tabard,"""The Tabard, 2 Bath Road, Chiswick""",W4 1LW,521258,Hounslow,51.495693,-0.254641,264,0.066086,0.786735,264.0,45.0,50.458417,NaN,48.8425,13.164601,62.007101
1727,10789,111961,The Raven,"""375 Goldhawk Road, London""",W6 0SA,521948,Hammersmith and Fulham,51.494137,-0.244752,241,0.124638,1.483780,241.0,50.0,56.155462,NaN,48.8425,14.861646,63.704147
1631,10693,110478,Belushi's And St Christopher's,"""Hammersmith Broadway, London""",W6 7AB,523351,Hammersmith and Fulham,51.493303,-0.224573,110,0.049227,0.586033,110.0,43.0,48.257716,43.917819,48.8425,14.963900,63.806400
1737,10799,111979,The Swan,"""46 Hammersmith Broadway, London""",W6 0DZ,523315,Hammersmith and Fulham,51.493131,-0.225098,110,0.052167,0.621035,110.0,43.0,48.292718,43.952821,48.8425,14.998902,63.841402
1742,10804,111987,The William Morris,"""2 - 4 King Street, London""",W6 0QA,523275,Hammersmith and Fulham,51.493085,-0.225676,110,0.069804,0.830995,110.0,43.0,48.502677,44.162781,48.8425,15.208861,64.051362
1628,10690,110466,Be At One,"""14A Hammersmith Broadway, London""",W6 7AL,523408,Hammersmith and Fulham,51.493336,-0.223751,110,0.097927,1.165803,110.0,43.0,48.837485,44.497589,48.8425,15.543669,64.386170
1710,10772,111934,The Hop Poles,"""Hop Poles, 17 - 19 King Street, London""",W6 9HR,523278,Hammersmith and Fulham,51.492753,-0.225646,110,0.101516,1.208529,110.0,43.0,48.880211,44.540314,48.8425,15.586395,64.428895
2272,11334,120211,The Roebuck,"""The Roebuck, 122 Chiswick High Road, Chiswick""",W4 1PU,521401,Hounslow,51.493022,-0.252674,264,0.270386,3.218881,264.0,45.0,52.890563,NaN,48.8425,15.596747,64.439248
2268,11330,120202,The Packhorse and Talbot,"""The Pack Horse And Talbot, 145 Chiswick High ...",W4 2DT,521231,Hounslow,51.492493,-0.255144,264,0.291602,3.471450,264.0,45.0,53.143132,NaN,48.8425,15.849316,64.691817
1729,10791,111966,The Rooks Nest,"""The Egerton, 73 Dalling Road, London""",W6 0JD,522635,Hammersmith and Fulham,51.494069,-0.234857,208,0.073684,0.877191,208.0,49.0,54.548873,NaN,48.8425,16.255057,65.097557


,index,0_x,1,2,3,4,8,latitude,longitude,0_y,0,traveltime_totube_pub,tube_id,travel_time_to_tube,total time1
0,1875,17820,The Three Jolly Wheelers,"""735 Chigwell Road, Woodford Green, Chigwell, ...",IG8 8AS,542653,Redbridge,51.608869,0.058699,214,1.372238,1.633617e+01,NaN,NaN,NaN
1,8950,85383,Aria Bar,"""31 Longbridge Road, Barking""",IG11 8TN,544481,Barking and Dagenham,51.540240,0.081962,15,0.097452,1.160141e+00,15.0,23.0,30.491926
2,8951,85412,Barking Arms Ltd,"""25-27 Station Parade, Barking""",IG11 8TU,544404,Barking and Dagenham,51.539531,0.080819,15,0.014687,1.748431e-01,15.0,23.0,29.506629
3,8952,85420,Barking football Club,"""Barking Football Club, Lodge Avenue, Dagenham""",RM8 2JR,546614,Barking and Dagenham,51.545377,0.112960,268,1.106634,1.317422e+01,NaN,NaN,NaN
4,8953,85423,Barking Indoor Bowls Club Ltd,"""Indoor Bowls Hall, Barking Prk, Longbridge Ro...",IG11 8TA,544692,Barking and Dagenham,51.542225,0.085096,15,0.406872,4.843717e+00,15.0,23.0,34.175503
5,8954,85429,Barking Rugby Union Football Club,"""Gale Street, Dagenham""",RM9 4TX,547647,Barking and Dagenham,51.534920,0.127357,21,0.598926,7.130071e+00,21.0,27.0,40.461857
6,8955,85436,Barking United Services Club,"""79 Blake Avenue, Barking""",IG11 9SF,545650,Barking and Dagenham,51.533030,0.098489,268,0.640896,7.629717e+00,NaN,NaN,NaN
7,8956,85440,Beacon Tree,"""Beacon Tree, 945 Green Lane, Dagenham""",RM8 1DJ,548489,Barking and Dagenham,51.561457,0.140758,67,2.238250,2.664583e+01,67.0,30.0,62.977618
8,8957,85519,Chadwell Heath Cricket Club,"""St Chads Park, West Road, Chadwell Heath""",RM6 6YA,547980,Barking and Dagenham,51.575822,0.134086,173,3.054550,3.636369e+01,173.0,30.0,72.695480
9,8958,85589,Dagenham Bowling,"""Dagenham Leisure Park, Cook Road, Dagenham""",RM9 6UQ,548237,Barking and Dagenham,51.532573,0.135769,21,1.052035,1.252423e+01,21.0,27.0,45.856012


In [11]:
person_travel

NameError: name 'person_travel' is not defined